### Imports

In [ ]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

import utils

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_COMPUTE_DID_1 = "did:nv:299d23f05257d26fcbad7b4871f50f03902dc81b03045813910fc8f8f84150aa"
ASSET_COMPUTE_DID_2 = "did:nv:68b7f6f44a0285f7e7d9eca35118d848141b95b408a1ce785f39cbb80d159fb1"
ASSET_FLOWER_SERVER_DID = "did:nv:ea05cae05e80c9d80b6cc8b289337f8a2059fb3b689cebf2f1995e24448d6981"

### Setup Nevermined and accounts

In [ ]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


### Publish algorithm

In [ ]:
metadata_algorithm_client = {
  "main": {
    "name": "Flower Demo Client",
    "dateCreated": utils.date_now(),
    "author": "Adap",
    "license": "",
    "price": "0",
    "files": [
      {
        "index": 0,
        "contentType": "text/text",
        "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
        "checksumType": "MD5",
        "contentLength": "12057507",
        "url": "https://github.com/nevermined-io/fl-demo/raw/feature/flower/notebooks/flwr_client.ipnby"
      }
    ],
    "type": "algorithm",
    "algorithm": {
      "language": "python",
      "format": "py",
      "version": "0.1.0",
      "entrypoint": (
          "pip install tensorflow-cpu numpy flwr==0.15 jupyter papermill && "
          "papermill --stdout-file - flwr_client.ipynb $NEVERMINED_OUTPUTS_PATH/flwr_client_output.ipynb"
      ),
      "requirements": {
        "container": {
          "image": "python",
          "tag": "3.8-slim-buster",
          "checksum": "sha256:53ad3a03b2fb240b6c494339821e6638cd44c989bcf26ec4d51a6a52f7518c1d"
        }
      }
    }
  }
}

utils.print_json(metadata_algorithm_client)

ddo_algorithm_client = nevermined.assets.create(metadata_algorithm_client, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm_client.did}")

In [ ]:
metadata_algorithm_server = {
  "main": {
    "name": "Flower Demo Server",
    "dateCreated": utils.date_now(),
    "author": "Adap",
    "license": "",
    "price": "0",
    "files": [
      {
        "index": 0,
        "contentType": "text/text",
        "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
        "checksumType": "MD5",
        "contentLength": "12057507",
        "url": "https://github.com/nevermined-io/fl-demo/raw/feature/flower/notebooks/flwr_server.ipnby"
      }
    ],
    "type": "algorithm",
    "algorithm": {
      "language": "python",
      "format": "py",
      "version": "0.1.0",
      "entrypoint": (
          "pip install tensorflow-cpu numpy flwr==0.15 jupyter papermill && "
          "papermill --stdout-file - flwr_server.ipynb $NEVERMINED_OUTPUTS_PATH/flwr_server_output.ipynb"
      ),
      "requirements": {
        "container": {
          "image": "python",
          "tag": "3.8-slim-buster",
          "checksum": "sha256:53ad3a03b2fb240b6c494339821e6638cd44c989bcf26ec4d51a6a52f7518c1d"
        }
      }
    }
  }
}

utils.print_json(metadata_algorithm_server)

ddo_algorithm_server = nevermined.assets.create(metadata_algorithm_server, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm_server.did}")

### Publish the workflows:

- Two for the compute to the data assets
- One for the flower service

In [ ]:
# Compute asset 1
metadata_workflow_client1 = {
  "main": {
    "name": "Flower Client",
    "dateCreated": utils.date_now(),
    "author": "Adap",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [
              {
                "index": 0,
                "id": ASSET_COMPUTE_DID_1
              }
          ],
          "transformation": {
            "id": ddo_algorithm_client.did
          }
        }
      ]
    }
  }
}

utils.print_json(metadata_workflow_client1)

ddo_workflow_client1 = nevermined.assets.create(metadata_workflow_client1, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_client1.did}")

In [ ]:
# Compute asset 2
metadata_workflow_client2 = {
  "main": {
    "name": "Flower Client",
    "dateCreated": utils.date_now(),
    "author": "Adap",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [
              {
                "index": 0,
                "id": ASSET_COMPUTE_DID_2
              }
          ],
          "transformation": {
            "id": ddo_algorithm_client.did
          }
        }
      ]
    }
  }
}

utils.print_json(metadata_workflow_client2)

ddo_workflow_client2 = nevermined.assets.create(metadata_workflow_client2, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_client2.did}")

In [ ]:
# Compute server
metadata_workflow_server = {
  "main": {
    "name": "Flower Server",
    "dateCreated": utils.date_now(),
    "author": "Adap",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [],
          "transformation": {
            "id": ddo_algorithm_server.did
          }
        }
      ]
    }
  }
}

utils.print_json(metadata_workflow_server)

ddo_workflow_server = nevermined.assets.create(metadata_workflow_server, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_server.did}")

### Order computations:

- One for each data asset
- One for the coordinator service

In between we want for the blockchain to process the request.

In [ ]:
keeper = Keeper.get_instance()

service_agreement_id1 = nevermined.assets.order(ASSET_COMPUTE_DID_1, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id1}")
utils.wait_for_event(keeper, service_agreement_id1)

service_agreement_id2 = nevermined.assets.order(ASSET_COMPUTE_DID_2, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id2}")
utils.wait_for_event(keeper, service_agreement_id2)

service_agreement_id_server = nevermined.assets.order(ASSET_FLOWER_SERVER_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id_server}")
utils.wait_for_event(keeper, service_agreement_id_server)


### Execute workflows

This orders the Nevermined to start the execution of the workflows:
- One for each data asset
- One for the coordinator service

In [ ]:
execution_id1 = nevermined.assets.execute(
    service_agreement_id1,
    ASSET_COMPUTE_DID_1,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_client1.did
)
print(f"Execution ID: {execution_id1}")

In [ ]:
execution_id2 = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_client2.did
)
print(f"Execution ID: {execution_id2}")

In [ ]:
execution_id_coordinator = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_FLOWER_SERVER_DID,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_server.did
)
print(f"Execution ID: {execution_id_coordinator}")

### Wait for compute jobs to finish

In [ ]:
succeeded = set()
failed = set()
outputs = set()
while len(succeeded) < 3:
    for job_id in [execution_id1, execution_id2, execution_id_coordinator]:
        result = nevermined.assets.compute_status(service_agreement_id1, job_id, consumer_account)
        status = result["status"]
        print(f"{job_id}: {status}")

        if status == "Succeeded":
            succeeded.add(job_id)
            outputs.add(result["did"])
        elif status == "Failed":
            raise Exception("Some jobs failed")
    print()
    time.sleep(10)

for did in outputs:
    print(f"Output DID: {did}")

### Download the results

In [ ]:
for did in outputs:
    print(f"Downloading: {did}")
    nevermined.assets.download(did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")